## 01 Data Cleaning

In [27]:
import pandas as pd
import sys

sys.path.append('../src')

car_df = pd.read_csv('../data/raw/car_prices.csv')
print("Initial DataFrame shape:", car_df.shape)
car_df.describe()

Initial DataFrame shape: (558836, 17)


/var/folders/25/myn29lpd1kgc8cf9vjv6822h0000gn/T/ipykernel_61443/3956737526.py:6: DtypeWarning: Columns (8,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  car_df = pd.read_csv('../data/raw/car_prices.csv')


,year,odometer,sellingprice
count,558836.000000,558742.000000,558836.000000
mean,2010.038918,68320.138936,13611.265715
std,3.966862,53398.513797,9749.439064
min,1982.000000,1.000000,1.000000
25%,2007.000000,28371.000000,6900.000000
50%,2012.000000,52254.000000,12100.000000
75%,2013.000000,99109.000000,18200.000000
max,2015.000000,999999.000000,230000.000000


- Only 3 columns display with .describe() because most car data is string type
- Maximum odometer is 999,999.0 which is likely incorrect 
- Min odometer and sellingprice of 1.0 are also oddities

In [28]:
print(car_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558836 entries, 0 to 558835
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          558836 non-null  int64  
 1   make          548535 non-null  object 
 2   model         548437 non-null  object 
 3   trim          548185 non-null  object 
 4   body          545641 non-null  object 
 5   transmission  493483 non-null  object 
 6   vin           558832 non-null  object 
 7   state         558836 non-null  object 
 8   condition     547042 non-null  object 
 9   odometer      558742 non-null  float64
 10  color         558087 non-null  object 
 11  interior      558087 non-null  object 
 12  seller        558836 non-null  object 
 13  mmr           558836 non-null  object 
 14  sellingprice  558836 non-null  int64  
 15  saledate      558836 non-null  object 
 16  Unnamed: 16   25 non-null      object 
dtypes: float64(1), int64(2), object(14)
memory usage

- Pandas shows column as object type when it contains strings, mixed types, or non-numeric values
- How to represent these values numericaly without one-hot encoding all?

In [29]:
car_df.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate,Unnamed: 16
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,"kia motors america, inc",20500,21500,Tue Dec 16 2014 12:30:00 GMT-0800 (PST),NaN
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,"kia motors america, inc",20800,21500,Tue Dec 16 2014 12:30:00 GMT-0800 (PST),NaN
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,4.5,1331.0,gray,black,financial services remarketing (lease),31900,30000,Thu Jan 15 2015 04:30:00 GMT-0800 (PST),NaN
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,4.1,14282.0,white,black,volvo na rep/world omni,27500,27750,Thu Jan 29 2015 04:30:00 GMT-0800 (PST),NaN
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,4.3,2641.0,gray,black,financial services remarketing (lease),66000,67000,Thu Dec 18 2014 12:30:00 GMT-0800 (PST),NaN


In [30]:
# Show percentage of nulls per column
car_df.isnull().mean().sort_values(ascending=False) * 100

Unnamed: 16     99.995526
transmission    11.694486
body             2.361158
condition        2.110458
trim             1.905926
model            1.860832
make             1.843296
color            0.134029
interior         0.134029
odometer         0.016821
vin              0.000716
state            0.000000
seller           0.000000
mmr              0.000000
sellingprice     0.000000
saledate         0.000000
year             0.000000
dtype: float64

- Unnamed column is 99.99% null and has to be dropped
- Transmission is 11.6% null and we can replace those values
- Should probably drop all the ones where make, model, or odometer are null

In [31]:
# Find the most frequent value in each column
# mode() returns a DataFrame; we select the first row with iloc[0]
most_frequent_values = car_df.mode().iloc[0]
print(most_frequent_values)

year                                               2012
make                                               Ford
model                                            Altima
trim                                               Base
body                                              Sedan
transmission                                  automatic
vin                                           automatic
state                                                fl
condition                                           1.9
odometer                                            1.0
color                                             black
interior                                          black
seller                               nissan-infiniti lt
mmr                                               12500
sellingprice                                      11000
saledate        Tue Feb 10 2015 01:30:00 GMT-0800 (PST)
Unnamed: 16     Tue Jun 02 2015 02:15:00 GMT-0700 (PDT)
Name: 0, dtype: object


In [32]:
car_df['transmission'] = car_df['transmission'].fillna('automatic')
car_df['interior'] = car_df['interior'].fillna('black')
car_df['color'] = car_df['color'].fillna('black')
car_df['trim'] = car_df['trim'].fillna('Base')

# Convert 'condition' column to numeric, coerce errors to NaN, then fill NaNs with the mean
car_df['condition'] = pd.to_numeric(car_df['condition'], errors='coerce')
car_df['condition'] = car_df['condition'].fillna(car_df['condition'].mean())

# Drop rows where a specific column has null values
car_df = car_df.dropna(subset=['make', 'model', 'odometer', 'body'])

# Drop unnecessary columns 
# 'vin' is a unique identifier, and 'Unnamed: 16' appears to be an extraneous column
car_df = car_df.drop(columns=['vin', 'Unnamed: 16', 'saledate', 'seller', 'trim'])

# Show percentage of nulls per column
print("Percentage of nulls per column after cleaning:")
print(car_df.isnull().mean().sort_values(ascending=False) * 100)

Percentage of nulls per column after cleaning:
year            0.0
make            0.0
model           0.0
body            0.0
transmission    0.0
state           0.0
condition       0.0
odometer        0.0
color           0.0
interior        0.0
mmr             0.0
sellingprice    0.0
dtype: float64


- Columns no longer have any null values
- Next question is how to encode all the string type columns

In [33]:
# Certain rows in 'transmission' feature have values other than 'automatic' or 'manual'
car_df['transmission'] = car_df['transmission'].replace(['sedan', 'Sedan'], 'automatic')
car_df = pd.get_dummies(car_df, columns=['transmission'])
print(f"Number of columns after one-hot encoding: {len(car_df.columns)}")
car_df.head()

Number of columns after one-hot encoding: 13


,year,make,model,body,state,condition,odometer,color,interior,mmr,sellingprice,transmission_automatic,transmission_manual
0,2015,Kia,Sorento,SUV,ca,5.0,16639.0,white,black,20500,21500,True,False
1,2015,Kia,Sorento,SUV,ca,5.0,9393.0,white,beige,20800,21500,True,False
2,2014,BMW,3 Series,Sedan,ca,4.5,1331.0,gray,black,31900,30000,True,False
3,2015,Volvo,S60,Sedan,ca,4.1,14282.0,white,black,27500,27750,True,False
4,2014,BMW,6 Series Gran Coupe,Sedan,ca,4.3,2641.0,gray,black,66000,67000,True,False


In [34]:
print(f"Number of unique colors: {car_df['color'].nunique()}")
print(f"Unique colors before cleaning: {car_df['color'].unique()}")
print('\n')

car_df['color'] = car_df['color'].replace('—', 'black')
car_df['color'] = car_df['color'].str.replace(r'^\d+$', 'black', regex=True)
print(f"Unique colors after cleaning: {car_df['color'].unique()}")

Number of unique colors: 45
Unique colors before cleaning: ['white' 'gray' 'black' 'red' 'silver' 'blue' 'brown' 'beige' 'purple'
 'burgundy' '—' 'gold' 'yellow' 'green' 'charcoal' 'orange' 'off-white'
 'turquoise' 'pink' 'lime' '9410' '1167' '2172' '14872' '12655' '15719'
 '6388' '16633' '11034' '2711' '6864' '339' '18384' '9887' '9837' '20379'
 '20627' '721' '6158' '2817' '5705' '18561' '2846' '9562' '5001']


Unique colors after cleaning: ['white' 'gray' 'black' 'red' 'silver' 'blue' 'brown' 'beige' 'purple'
 'burgundy' 'gold' 'yellow' 'green' 'charcoal' 'orange' 'off-white'
 'turquoise' 'pink' 'lime']


In [35]:
print(f"Number of unique states: {car_df['state'].nunique()}")
print(f"Unique states before cleaning: {car_df['state'].unique()}")
print('\n')

car_df['state'] = car_df['state'].str.replace(r'^.{3,}$', 'unknown', regex=True)
print(f"Unique states after cleaning: {car_df['state'].unique()}")

Number of unique states: 63
Unique states before cleaning: ['ca' 'tx' 'pa' 'mn' 'az' 'wi' 'tn' 'md' 'fl' 'ne' 'nv' 'oh' 'mi' 'nj'
 'ga' 'va' 'sc' 'nc' 'in' 'il' 'co' 'ut' 'mo' 'ny' 'ma' 'pr' 'or' 'la'
 'wa' 'hi' 'qc' 'ab' 'on' 'ok' 'ms' 'nm' 'al' 'ns' '3vwd17aj2fm258506'
 '3vwd17aj3fm276741' '3vwd17aj2fm285365' '3vwd17aj0fm227318'
 '3vwd17aj6fm218641' '3vwd17aj7fm223475' '3vwd17aj5fm297123'
 '3vwd17aj5fm219943' '3vwd17aj9fm219766' '3vwd17aj3fm259017'
 '3vwd17aj5fm206111' '3vwd17aj5fm273601' '3vwd17aj5fm221322'
 '3vwd17aj5fm268964' '3vwd17aj6fm231972' '3vwd17aj7fm222388'
 '3vwd17aj7fm218440' '3vwd17ajxfm315938' '3vwd17aj7fm229552'
 '3vwd17aj8fm298895' '3vwd17aj4fm236636' '3vwd17aj5fm225953'
 '3vwd17aj7fm326640' '3vwd17aj8fm239622' '3vwd17aj2fm261566']


Unique states after cleaning: ['ca' 'tx' 'pa' 'mn' 'az' 'wi' 'tn' 'md' 'fl' 'ne' 'nv' 'oh' 'mi' 'nj'
 'ga' 'va' 'sc' 'nc' 'in' 'il' 'co' 'ut' 'mo' 'ny' 'ma' 'pr' 'or' 'la'
 'wa' 'hi' 'qc' 'ab' 'on' 'ok' 'ms' 'nm' 'al' 'ns' 'unknown']


In [36]:
print(f"Number of unique bodies: {car_df['body'].nunique()}")
print(f"Unique bodies before cleaning: {car_df['body'].unique()}")
print('\n')

# created a custom script to check for keywords to standardize body types
from cleaning_scripts import standardize_body

car_df['body'] = car_df['body'].apply(standardize_body)
print(f"Unique bodies after cleaning: {car_df['body'].unique()}")

Number of unique bodies: 87
Unique bodies before cleaning: ['SUV' 'Sedan' 'Convertible' 'Coupe' 'Wagon' 'Hatchback' 'Crew Cab'
 'G Coupe' 'G Sedan' 'Elantra Coupe' 'Genesis Coupe' 'Minivan' 'Van'
 'Double Cab' 'CrewMax Cab' 'Access Cab' 'King Cab' 'SuperCrew'
 'CTS Coupe' 'Extended Cab' 'E-Series Van' 'SuperCab' 'Regular Cab'
 'G Convertible' 'Koup' 'Quad Cab' 'CTS-V Coupe' 'sedan' 'G37 Convertible'
 'Club Cab' 'Xtracab' 'Q60 Convertible' 'CTS Wagon' 'convertible'
 'G37 Coupe' 'Mega Cab' 'Cab Plus 4' 'Q60 Coupe' 'Cab Plus'
 'Beetle Convertible' 'TSX Sport Wagon' 'Promaster Cargo Van'
 'GranTurismo Convertible' 'CTS-V Wagon' 'Ram Van' 'minivan' 'suv'
 'Transit Van' ' Navitgation' 'van' 'regular-cab' 'g sedan' 'g coupe'
 'hatchback' 'king cab' 'supercrew' 'g convertible' 'coupe' 'crew cab'
 'wagon' 'double cab' 'e-series van' 'regular cab' 'quad cab'
 'g37 convertible' 'supercab' 'extended cab' 'crewmax cab' 'genesis coupe'
 'access cab' 'mega cab' 'xtracab' 'beetle convertible' 'cts cou

In [37]:
print(f"Number of unique models: {car_df['model'].nunique()}")
print(f"Unique models: {car_df['model'].unique()}")
print('\n')

print(f"Number of unique makes: {car_df['make'].nunique()}")
print(f"Unique makes: {car_df['make'].unique()}")
print('\n')

Number of unique models: 776
Unique models: ['Sorento' '3 Series' 'S60' '6 Series Gran Coupe' 'Altima' 'M5' 'Cruze'
 'A4' 'Camaro' 'A6' 'Optima' 'Fusion' 'Sonata' 'Q5' '6 Series' 'Impala'
 '5 Series' 'A3' 'XC70' 'X5' 'SQ5' 'S5' 'Verano' 'Suburban' 'ELR' 'V60'
 'X6' 'ILX' 'K900' 'Malibu' 'RX 350' 'Versa' 'Elantra' 'Versa Note' 'A8'
 'X1' 'Enclave' 'TTS' '4 Series' 'Silverado 2500HD' 'MDX' 'Silverado 1500'
 'SRX' 'G Coupe' 'G Sedan' 'FX' 'Santa Fe' 'Genesis' 'Equus'
 'Sonata Hybrid' 'Accent' 'Veloster' 'Elantra Coupe' 'Azera' 'Tucson'
 'Genesis Coupe' 'Wrangler' 'S-Class' 'GS 350' 'Outlander' 'C-Class'
 'Mazda2' 'Rio' 'M' '370Z' 'Soul' 'Outlander Sport' 'SLK-Class' 'ES 350'
 'E-Class' 'Mazda3' 'Cooper Clubman' 'Cooper' 'CX-9' 'Forte' 'Compass'
 'JX' 'RX 450h' 'LR4' 'Mazda5' 'Range Rover Evoque' 'LS 460' 'GLK-Class'
 'Sportage' 'Grand Cherokee' 'MKX' 'XF' 'GL-Class' 'M-Class'
 'Cooper Countryman' 'Lancer' 'Range Rover Sport' 'Passat' 'Corolla'
 'XC60' 'Sienna' 'Juke' 'Yaris' 'Sentra' 'Rog

In [38]:
# one-hot encode the remaining categorical features
car_df = pd.get_dummies(car_df, columns=['make', 'model', 'body', 'color', 'interior', 'state'])
car_df.head()


,year,condition,odometer,mmr,sellingprice,transmission_automatic,transmission_manual,make_Acura,make_Aston Martin,make_Audi,...,state_pr,state_qc,state_sc,state_tn,state_tx,state_unknown,state_ut,state_va,state_wa,state_wi
0,2015,5.0,16639.0,20500,21500,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2015,5.0,9393.0,20800,21500,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2014,4.5,1331.0,31900,30000,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2015,4.1,14282.0,27500,27750,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2014,4.3,2641.0,66000,67000,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [39]:
print(car_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 545458 entries, 0 to 558835
Columns: 920 entries, year to state_wi
dtypes: bool(915), float64(2), int64(2), object(1)
memory usage: 500.9+ MB
None


- One column is still considered object
- All other columns are either numeric or bool

In [40]:
print("Object (string) columns:")
print(car_df.select_dtypes(include='object').columns)

car_df['mmr'] = pd.to_numeric(car_df['mmr'], errors='coerce')

print(f"Number of unique mmrs: {car_df['mmr'].nunique()}")

Object (string) columns:
Index(['mmr'], dtype='object')
Number of unique mmrs: 1101


In [41]:
print(car_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 545458 entries, 0 to 558835
Columns: 920 entries, year to state_wi
dtypes: bool(915), float64(3), int64(2)
memory usage: 500.9 MB
None


- Now the data can be saved into a csv for clean data

In [43]:
# Save cleaned data to CSV
car_df.to_csv('../data/cleaned/car_prices_cleaned.csv', index=False)
print("Data saved to '../data/cleaned/car_prices_cleaned.csv'")

Data saved to '../data/cleaned/car_prices_cleaned.csv'
